# Bienvenidos!!! 

### Esta Notebook tiene como objetivo que vean un ejemplo sencillo de un proceso bayesiano usado para AB testing. Van a tener la oportunidad de jugar con los datos para decidir que conjuntos de datos vamos a comparar para que puedan ver posteriormente el resultado de nuestro proceso.

### Primero un par de aclaraciones:
* No hace falta que sepan programar, con que ejecuten **RUN, RUN, RUN** van a obtener los resultados.
* No hace falta que toquen el codigo para ajustar parametros, una vez ejecutado el codigo les van a aparecer **widgets** (Barritas) para seleccionar los parametros que quieran.
* **Vamos a ver juntos el paso a paso de este proceso**, no se frusten si no entienden algo en su totalidad,  especialmente el armado del modelo jerárquicos (Después aclaramos que es), ya que esto representa una de las grandes complejidades de este tipo de modelos y dependen del Data Scientist. De cualquier forma, si les interesa el tema **los animamos a que exploren mas, ante cualquier consulta les dejamos nuestros contactos**.


### Esperamos que les sea útil esta experiencia y les brindamos abajo un medio de comunicación por cualquier duda!
#### Alejandro Daniel Attento alejandro.attento@gmail.com [LinkedIn](https://www.linkedin.com/in/alejandro-daniel-attento/)

### Que lo disfruten y arrancamos!

## ¿Qué vamos a hacer?

Acaso no es esto lo que todos nos preguntamos? Bueno, si recuerdan la clase que dimos recordaran que explicamos un ejemplo de AB testing con una métrica que solemos usar mucho, **ARPDAU** *(Average Revenue per Daily Active User)*, seguro la recuerdan. Básicamente es el ingreso promedio por jugador que tenemos a diario, es una métrica que se suelen utilizar en varias industrias, si no la conocían les podría ser útil en el futuro.

**Vamos a suponer que planteamos un cambio en nuestro producto** que puede ser una aplicación móvil. 

**Ya paso un tiempo y tenemos los datos de nuestra prueba**, tenemos dos conjuntos, el primero sobre un grupo de testeo al que se le aplico el cambio y el otro sobre un grupo al que no se le aplico este cambio, a este grupo se lo llama de control. De nuestra evaluación depende que la empresa tome la decisión de implementar un cambio en nuestro producto o no *(Todos los ojos están puestos en nosotros)*.

Lo que vamos a hacer a lo largo de esta notebook es realizar esta prueba, la idea es que entiendan que **todo esto es simplemente matemática**, no hay magia, no hay nada que no pudieran entender si les interesa y se sientan a estudiarlo, todos arrancamos de 0 alguna vez :).

Vamos a explicarles cada paso del proceso, no nos vamos a meter en el detalle técnico o matemático, esto daría para un curso entero aparte. **Vamos a explicar que se tiene en cuenta y la forma que tienen estos modelos, así cuando en el futuro se encuentren con uno tengan una idea de que se trata**.

## ¿Por qué bayesiano, que tiene de especial?

Resumiendo, mucho lo que vimos, una prueba de AB testing bayesiana toma los conjuntos de datos que vamos a comparar, se los estudia y se establece un modelo de distribuciones que se relacionan entre sí, esto nos permiten generar más muestras a partir de la cantidad que tenemos inicialmente. 

Es decir, si tras correr la prueba dos meses tenemos 60 muestras para cada grupo *(Una por día)*, inferimos la distribución probabilística que tienen estas muestras y generamos más, supongamos 1.000, 100.000, 500.000 eso depende de distintos factores que ahora no importan. Lo que si deben tener en mente es que generamos **MUCHAS** muestras, lo que nos permite obtener las distribuciones de cada conjunto para compararlas.

## ¿Por qué no comparamos directamente las distribuciones de las 60 muestras iniciales?

La realidad es que es una cantidad de muestras muy baja, lo que implica que no podemos asegurar que comparar ambas distribuciones sea algo valido. Decimos del lado de Data Science que **tendríamos una alta Incertidumbre**. Por eso mismo usamos un método bayesiano, inferimos las distribuciones que tendría cada conjunto para generar una cantidad mayor de muestras y compararlas.

# Hasta acá una breve introducción, arranquemos con lo que nos interesa.

## 1. Preparativos

En general, cuando trabajemos con Python vamos a necesitar de determinadas librerías que nos brindan diferentes funciones que necesitamos, en este caso para performar el AB test bayesiano. Para asegurarnos que no tengan ningún tipo de problema les dejo abajo un listado con las librerías que usamos y sus versiones *(Si usan las librerías con la misma versión o una superior no deberían de tener problemas, si tienen alguna versión más antigua les recomendamos que la actualicen)*:

* **Numpy** V1.16.5 *Nos brinda funciones numéricas para trabajar con vectores y matrices*
* **Pandas** V0.25.1 *Nos brinda funciones para trabajar con conjuntos de datos, lo que solemos llamar dataframes*
* **Pymc3** V3.8 *Nos brinda funciones que nos permite realizar el procesos bayesianos*
* **Plotly** V4.12.0 *Nos brinda funciones para realizar gráficos*
* **Matplotlib** V3.1.1 *Nos brinda funciones para realizar gráficos*
* **ipywidgets** V7.5.1 *Nos brinda funciones para generar widgets*


Tras ejecutar la próxima celda van a ver las versiones que les cargo de cada librería  Anaconda *(El entorno en el cual estamos trabajando)*, de aparecerles algún error por no tener una librería o estar desactualizada sigan los pasos del siguiente [video](https://www.youtube.com/watch?v=7_o_kjG1Sfs).

Noten que usamos más de una libreria para hacer graficos, existen varias que se usan actualmente, esto es simplemente para que vean que se puede lograr lo mismo con distintas librerias *(Incluso muchas veces unas se basan en otras)*.

In [ ]:
# 1.1 Importar librerías
import numpy as np
import pandas as pd
import pymc3 as pm
import plotly.graph_objects as go
from plotly import subplots
import plotly
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as iwg
import warnings

# 1.2 Ignora mensajes de advertencias
warnings.filterwarnings('ignore')

# 1.3 Muestra las versiones de las librerías que estamos usando
print("""Version usadas:\nNumpy: {}\nPandas: {}\nPymc3: {}\nPlotly: {}\nIpywidgets: {}\nMatplotlib: {}"""\
      .format(np.__version__, pd.__version__, pm.__version__, ply.__version__, iwg.__version__, matplotlib.__version__))

## 2. Juguemos  un poco con los datos

Ver el resultado del proceso bayesiano sin haber jugado antes un poco con los datos que usamos no tiene sentido, no les va a servir de nada. Corran la celda de abajo, se les van a generar 4 widgets para poder ajustar la media y el desvío estándar tanto del grupo de control como del grupo de testing *(Vamos a generar ambos conjuntos de datos a partir de una distribución normal para nuestro ejemplo)*.

La idea es que vean la serie temporal y el histograma de los datos que ustedes generen, de esta forma pueden ver que tan parecidos o no serían los conjuntos que vamos a evaluar, esto les va a servir para entender mejor el resultado final.

In [ ]:
# 2.1 Generamos las fechas entre el 2020-06-01 y el 2020-11-01
date_LST = [date.strftime('%m/%d/%Y') for date in pd.date_range('2020-06-01','2020-11-01',freq='d')]

In [ ]:
# 2.2 Hacemos nuestro grafico interactivo
# 2.3 Definimos la función de nuestros graficos interactivos
@iwg.interact
def SHOW_ARPDAU_TIMELINE(mean_cont=(0.5, 2, 0.01), sd_cont=(0, 0.2, 0.01),\
                         mean_test=(0.5, 2, 0.01), sd_test=(0, 0.2, 0.01)):
    
    # 2.3.1 Indicamos que graficamos dos graficos
    fig = subplots.make_subplots(rows=2, cols=1)
    
    # 2.3.2 Trazamos la serie temporal para el grupo de Control
    fig.add_trace(go.Line(name="Control", x=date_LST,\
                          y=[val if val >= 0 else 0 for val in np.random.normal(mean_cont, sd_cont, len(date_LST))],\
                          marker=dict(color='#d2691e')), row=1, col=1)
    
    # 2.3.3 Trazamos la serie temporal para el grupo de Testing
    fig.add_trace(go.Line(name="Testing", x=date_LST,\
                          y=[val if val >= 0 else 0 for val in np.random.normal(mean_test, sd_test, len(date_LST))],\
                          marker=dict(color='#6495ed')), row=1, col=1)
    
    # 2.3.4 Generamos el histograma para el grupo de Control
    fig.add_trace(go.Histogram(name="Control",\
                               x=[val if val >= 0 else 0 for val in np.random.normal(mean_cont, sd_cont, len(date_LST))],\
                                   histnorm='probability', marker=dict(color='#d2691e')), row=2, col=1)
    
    # 2.3.5 Generamos el histograma para el grupo de Testing
    fig.add_trace(go.Histogram(name="Testing",\
                               x=[val if val >= 0 else 0 for val in np.random.normal(mean_test, sd_test, len(date_LST))],\
                               histnorm='probability', marker=dict(color='#6495ed')), row=2, col=1)   
    
    # 2.3.6 Definimos unos parametros de diseño, así como los titulos, etc.
    fig.update_traces(opacity=0.75)
    fig.update_layout(title='"Control vs. Testing Data" (ARPDAU)',
                      legend_title="Groups",
                      barmode='overlay',
                      font=dict(size=13),
                      height = 1000)

    # 2.3.7 Mostramos los graficos
    fig.show()

## 3. Vamos a crear los datos para el proceso

Si corren la celda que viene les aparecen los widgets para seleccionar la media y el desvío estándar de cada grupo de los datos que van a utilizar para el proceso bayesiano. Después ejecuten la celda a continuación para que se generen los datos finalmente.

In [ ]:
# 3.1 Genera widgets para la media del grupo de Control
mean_cont_WG = iwg.FloatSlider(value=0.65, min=0.5, max=2, step=0.01, readout_format='.2f',\
                                   description='Cont. mean', continuous_update=True)

# 3.2 Genera widgets para el desvio estandard del grupo de Control
sd_cont_WG = iwg.FloatSlider(value=0.08, min=0, max=0.2, step=0.01, readout_format='.2f',\
                                description='control. sd.', continuous_update=True)
    
# 3.3 Genera widgets para la media del grupo de Testing
mean_test_WG = iwg.FloatSlider(value=0.60, min=0.5, max=2, step=0.01, readout_format='.2f',\
                                   description='Test. mean', continuous_update=True)

# 3.4 Genera widgets para el desvio estandard del grupo de Testing
sd_test_WG = iwg.FloatSlider(value=0.06, min=0, max=0.2, step=0.01, readout_format='.2f',\
                                description='Test. sd', continuous_update=True)

# 3.5 Mostramos los widgets
display(mean_cont_WG, sd_cont_WG, mean_test_WG, sd_test_WG)

In [ ]:
# 3.6 Generamos las muestras según los parametros indicados para en grupo de Control y el de Testing
control_data_LST = [val if val >= 0 else 0 for val in np.random.normal(mean_cont_WG.value, sd_cont_WG.value, len(date_LST))]
testing_data_LST = [val if val >= 0 else 0 for val in np.random.normal(mean_test_WG.value, sd_test_WG.value, len(date_LST))]

## 4. Definimos el modelo jerárquico

Antes que nada recordemos dos o tres cositas:
* Cuando realizamos la inferencia bayesiana hay 3 elementos que necesitamos para esto:
 * La Prior *(La información histórica que tenemos de los datos en formato de distribuciones probabilísticas)*
 * La Verosimilitud *(La distribución probabilística que describe nuestra métrica)*
 * Los datos *(Todos aquellos datos que recopilamos a lo largo del AB test, en este caso los datos que ustedes generaron)*
* Cuando no tenemos información histórica de la métrica podemos usar una prior **poco informativas**, es decir, que tengan poco peso en la distribución final que obtenemos. ¿Cómo logramos esto? Asignándole una varianza grande por medio de los parámetros de la distribución, eso es lo que hacemos en este caso.
* El conjunto de la verosimilitud y las priors es lo que llamamos **el modelo jerárquico**. Un conjunto de distribuciones que definen el modelo estadístico que vamos a utilizar para la inferencia bayesiana.

El determinar la distribución para la verosimilitud y las priors es complejo y es uno de los trabajos más difíciles al trabajar con este tipo de modelos, esto es lo que llamamos el **modelado de las distribuciones**. No se preocupen por entender cómo se define esto, esto requiere una explicación mucho más extensa y una carga teórica mayor.


**En la primera celda vamos a definir los parámetros para las distribuciones prior**. Según la distribución de cada caso, si calculamos la varianza van a ver que es grande, como anticipamos ya que la idea es que tengan un peso bajo al no tener información histórica.

**En la segunda celda definimos el modelo jerárquico**, abajo se lo describimos brevemente de forma informativa, nuevamente no se preocupen por cómo se hace para determinar las distribuciones.

Recuerden que partimos de una Verosimilitud que en este caso está definida por una distribución t-student, esta tiene 3 parámetros *(A desempolvar libros de estadísticas)*, estas son:
* La Media
* El Desvío Estándar
* Los grados de libertad

Cada uno de estos parámetros tiene una prior que la definen, es decir, cada parámetros de la verosimilitud surge de otra distribución probabilística que llamamos prior, abajo tienen la definición completa del modelo jerárquico.
___________________________________________________________________________________________________________________________
**Verosimilitud: Distribución T-Student (med, sd, gl)** 

* **Media ~ Distribución Normal**
* **Desvío Estándar  ~ Distribución Gamma-Inversa**
* **Grado de Libertad ~ Distribución Exponencial**

*(med->Media, sd->Desvío Estándar, gl->Grados de Libertad)*

Finalmente le indicamos que vamos a usar los datos que definimos anteriormente
___________________________________________________________________________________________________________________________

Por los parámetro iniciales que le asignamos a las prior las distribuciones tienen las siguientes medias y varianzas:
* **Distribución Normal**: Media = 0 / Varianza = 100
* **Distribución Gamma Inversa**: Media = 1 / Varianza = 100
* **Diatribución Exponencial**: Media = 10 / Varianza = 100

In [ ]:
# 4.1 Definimos diccionarios para los parametros
norm_dist_DICT = {}
igamma_dist_DICT = {}
exp_dist_DICT = {}

# 4.2 Definimos parametros para la prior Normal
norm_dist_DICT['control_mean'] = 0
norm_dist_DICT['control_sd'] = 10
norm_dist_DICT['testing_mean'] = 0
norm_dist_DICT['testing_sd'] = 10

# 4.3 Definimos parametros para la prior Gamma Inversa
igamma_dist_DICT['control_alpha'] = 2.01
igamma_dist_DICT['control_beta'] = 1.01
igamma_dist_DICT['testing_alpha'] = 2.01 
igamma_dist_DICT['testing_beta'] = 1.01

# 4.3 Definimos parametros para la prior Exponencial
exp_dist_DICT['control_lambda'] = 0.1
exp_dist_DICT['testing_lambda'] = 0.1

In [ ]:
# 4.4 Definimos el modelo jerárquicos
def AB_tester(test_data, control_data, norm_dist_DICT, igamma_dist_DICT, exp_dist_DICT):
    with pm.Model() as model:
        # 4.4.1 Definimos prior de las Medias
        test_mean = pm.Normal('test_mean', mu=norm_dist_DICT['testing_mean'], sd=norm_dist_DICT['testing_sd'])
        control_mean = pm.Normal('control_mean', mu=norm_dist_DICT['control_mean'], sd=norm_dist_DICT['control_sd'])

        # 4.4.2 Definimos prior del Desvío Estándar
        test_std = 1/pm.Gamma('test_std', alpha=igamma_dist_DICT['testing_alpha'], beta=igamma_dist_DICT['testing_beta'])
        control_std = 1/pm.Gamma('control_std', alpha=igamma_dist_DICT['control_alpha'], beta=igamma_dist_DICT['control_beta'])

        # 4.4.3 Transformamos el Desvío Estándar en Precisión (Es otra forma de indicar el Desvío Estándar)
        test_prec = test_std**-2
        control_prec = control_std**-2

        # 4.4.4 Definimos prior de los Grados de Libertad
        test_dof = pm.Exponential('testing_dof', 1/exp_dist_DICT['testing_lambda'])
        control_dof = pm.Exponential('control_dof', 1/exp_dist_DICT['control_lambda'])

        # 4.4.5 Definimos la Verosimilitud
        test_group = pm.StudentT('test_group', nu=test_dof, mu=test_mean, lam=test_prec, observed=test_data)
        control_group = pm.StudentT('control_group', nu=control_dof, mu=control_mean, lam=control_prec, observed=control_data)
    
    # 4.4.6 Devolvemos el modelo jerárquicos definido
    return model

## 5. Realizamos la Inferencia Bayesiana

**Bueno gente ¡el gran momento!**

Vamos a realizar nuestra primera inferencia bayesiana. Esto va a ser sencillo, no se preocupen *(Ya se lo resolvimos nosotros)*, lo que hacemos es:
* En la próxima celda asignamos todo ese modelo jerárquico que armamos a una variable
* En la segunda celda corremos la inferencia bayesiana, como ven, le estamos aplicando unos parámetros. Lo que estamos haciendo es indicarle al proceso que:
 * Corra este proceso en paralelo 5 veces, los Data Scientist decimos que **corremos 5 cadenas en paralelo**. Esto es para asegurarnos que todos los casos convergen a un mismo valor, caso contrario nuestro modelo jerárquico no es el adecuado. Imagínese que si tras cada corrida el modelo converge a valores distintos, cada vez que lo corramos vamos a obtener un resultado distinto.
 * **Que genere 50000 muestras para cada grupo y cada cadena**, a la vez que descarte las primeras 5.000, ya que estos son muy erraticos y son lo que necesita el modelo para empezar a converger.
 * Que haga una verificación analítica de la convergencia de las cadenas entre sí.
 
El tema de la convergencia de las cadenas es muy importante, es algo que se mira mucho y se analiza analíticamente, abajo vamos a mostrarlo gráficamente para que se puedan dar una idea.


La inferencia bayesiana puede tardar un poco, esto depende de su computadora, acá estamos generando una baja cantidad de muestras para que no tengan que esperar demasiado. No debería de demorarse más de 5 minutos a lo sumo.

In [ ]:
# 5.1 Usamos la función del modelo jerarquico que creamos anteriormente
sc_model = AB_tester(testing_data_LST, control_data_LST, norm_dist_DICT, igamma_dist_DICT, exp_dist_DICT)

In [ ]:
# 5.2 Procesamos la inferencia bayesiana *(Puede demorar unos minutos)*
with sc_model:
    trace = pm.sample(50000, chains=5, tune=5000, discard_tuned_samples=True, compute_convergence_checks=True, progressbar=True, model=sc_model)

## 6. Analizamos la convergencia del proceso bayesiano

Vamos a obtener 6 filas de gráficos con dos gráficos cada uno, estos corresponden a **la media, el desvío estándar y los grados de libertado para el grupo de control y testeo**. 

El primer gráfico de cada fila muestra **la distribución final de cada una de las cadenas**, vamos a ver que todas las cadenas arrojan distribuciones muy muy parecidas *(Si no nos mandamos una macana)*, el segundo gráfico es **el trazado**, simplificándolo, si hubiese una cadena que converge a otro valor deberíamos de ver al menos una línea que resalta a simple vista, un patrón no regular.

**Si todo salió bien, nada de esto debio de haber ocurrido.**

Estos gráficos son para que se den una idea visual, en la práctica se hacen análisis analíticos para determinar si las cadenas realmente convergieron o no.

In [ ]:
# 6.1 Graficar el trazado de las cadenas
pm.traceplot(trace, var_names=['control_mean', 'test_mean', 'control_std', 'test_std', 'control_dof', 'testing_dof'],\
             figsize=(50,50), legend=True, compact=False)
display()

## 7. Analizamos el resultado de nuestro AB test

Ejecutando la próxima celda calculamos tanto para el grupo de testeo como para el grupo de control los percentiles 2.5, 50 (Esta es la mediana) y 97.5, además de las diferencias relativa porcentual de las cotas exteriores, de las cotas interiores y de las medianas.

Aclaramos para refrescar de la clase:
* **Cotas exteriores**: Las dos cotas, de cada uno de los grupos, que más se distancian entre sí.
* **Cotas interiores**: Las dos cotas, de cada uno de los grupos, que más cerca están entre sí.

Todo esto es lo que vamos a graficar ejecutando la última celda.
Recordemos que, si los intervalos de credibilidad que calculamos con los percentiles no se superponen la diferencia es significativa, caso contrario no lo es.

La fórmula de la diferencia relativa porcentual$$\triangle_{rel.} = \frac{x_{1}-x_{2}}{x_{1}}$$

In [ ]:
# 7.1 Calculamos los percentiles 2.5, 50, 97.5 para el grupo de Control
hpd_25_control = np.percentile(trace['control_mean'], 2.5)
hpd_500_control = np.percentile(trace['control_mean'], 50)
hpd_975_control = np.percentile(trace['control_mean'], 97.5)

# 7.2 Calculamos los percentiles 2.5, 50, 97.5 para el grupo de Testeo
hpd_25_test = np.percentile(trace['test_mean'], 2.5)
hpd_500_test = np.percentile(trace['test_mean'], 50)
hpd_975_test = np.percentile(trace['test_mean'], 97.5)

# 7.3 Calculamos la diferencia relativa porcentual entre las cotas exteriores, las interiores y las medianas.
bound1_per_diff = (hpd_25_control - hpd_975_test)/hpd_25_control
bound2_per_diff = (hpd_975_control - hpd_25_test)/hpd_975_control
median_per_diff = (hpd_500_control - hpd_500_test)/hpd_500_control

In [ ]:
# 7.4 Graficamos las distribuciones con los intervalos
fig, ax = plt.subplots(1)
fig.set_size_inches(15, 6)

# 7.4.1 Establecemos los graficos para el grafico de las distribuciones de la media
ax.set_title('Means', fontdict={'weight':'bold'})
ax.hist(trace['test_mean'], bins=100, color='b', alpha=0.5)
ax.hist(trace['control_mean'], bins=100, color='r', alpha=0.5)
ax.legend(['Test', 'Control'])
ax.axvline(x=hpd_25_control, ymin=0, ymax=2000, color='r')
ax.text(hpd_25_control,600,hpd_25_control.round(2), color='r', rotation=60)
ax.axvline(x=hpd_975_control, ymin=0, ymax=2000, color='r')
ax.text(hpd_975_control,600,hpd_975_control.round(2), color='r', rotation=60)
ax.axvline(x=hpd_25_test, ymin=0, ymax=2000, color='b')
ax.text(hpd_25_test,600,hpd_25_test.round(2), color='b', rotation=60)
ax.axvline(x=hpd_975_test, ymin=0, ymax=2000, color='b')
ax.text(hpd_975_test,600,hpd_975_test.round(2), color='b', rotation=60)

# 7.4.2 Establecemos los textos para el grafico de las distribuciones de la media
ax.text(min([hpd_25_control, hpd_25_test]), 7000,'Cotas 1: {:.2f}%'.format(bound1_per_diff*100), size=13)
ax.text(min([hpd_25_control, hpd_25_test]), 6500,'Cotas 2: {:.2f}%'.format(bound2_per_diff*100), size=13)
ax.text(min([hpd_25_control, hpd_25_test]), 6000,'Median:  {:.2f}%'.format(median_per_diff*100), size=13)

if (bound1_per_diff / abs(bound1_per_diff)) == (bound2_per_diff / abs(bound2_per_diff)):
    ax.text(min([hpd_25_control, hpd_25_test]), 7500,'La diferencia es significativa',\
               size=13, weight='bold', color = 'green')
else:
    ax.text(min([hpd_25_control, hpd_25_test]), 7500,'La diferencia NO es significativa',\
               size=13, weight='bold', color = 'red')
# 7.4.2 Mostramos el gráfico
fig.show()

## Lo logramos!

Finalmente corrieron su primer AB test usando estadística bayesiana  juntos, seguro que no muchos pueden decir eso. Esto es un breve resumen que toca los temas más importantes, omitimos muchos de los controles que haríamos en el proceso para asegurar que se cumplen todos los supuestos de estos modelos, los mencionamos porque es importante que sepan que existen.

Si les interesa el tema los animamos a averiguar e investigar más sobre el tema, estamos abiertos a cualquier consulta que tengan y esperamos que esta experiencia les haya sido útil. El código les queda de referencia, para aquellos de ustedes que sepan programar o que quieran aprender van a poder aprovecharlo, ¡siempre se arranca con algo! También pueden experimentar corriendo el modelo con distintos parámetros modificando los widgets para ver qué resultados obtienen.

¡Gracias por su tiempo y hasta la próxima!